In [2]:
import requests
from bs4 import BeautifulSoup
from bs4.element import ResultSet, Tag
import pandas as pd
from tabulate import tabulate
import csv
import time

In [3]:
base_url = "https://www.etoos.com/teacher/Evaluate/default.asp?"
teacher_id_list = [200180, 200439]
page_range = 34
# -> 총 데이터 개수 = 선생 수 x 페이지 범위 x 한 페이지 당 개수 = 2 x 34 x 15 = 1020
teacher_prefix = "TEACHER_ID="
page_prefix = "&page="
path = "./data/"
f_name = 'review.csv'
csv_full_name = path + f_name

def getFullUrl(teacher_id:int, page:int)->str:
    global base_url
    url = base_url + teacher_prefix + str(teacher_id) + page_prefix + str(page)
    return url

In [7]:
review_list = []
for teacher_id in teacher_id_list:
    for page in range(1,page_range + 1):
        print("curr page:", page)
        # 너무 자주보내면 응답을 안 줌
        try:
            response = requests.get(getFullUrl(teacher_id=teacher_id, page=page), timeout=5)
            time.sleep(2)
            # 응답 성공
            if response.status_code == 200:
                # 한글 깨짐 방지 인코딩
                response.encoding = 'euc-kr'
                soup = BeautifulSoup(response.text, 'html.parser')
                # 첫 페이지면 선생님 이름 가져오기
                if page == 1:
                    teacher_name = soup.select_one("div#TeacherLeftGnbEach a.tit").get_text(strip=True)
                    print(teacher_name)
                # 리뷰가 적힌 행들 가져오기
                rows = soup.select('tbody > tr')
                for row in rows:
                    # 등록 번호 찾기
                    reg_num = row.select_one('td:nth-child(1)').get_text(strip=True)
                    # BEST 수강 후기는 매 페이지마다 표시되므로 스킵
                    if reg_num == "BEST": continue
                    # 리뷰 내용 찾기
                    review_text = row.select_one('td.wr_tit p.tit')
                    ## 쓸데없는 정보 제거( new! 표시 )
                    if review_text.span != None:
                        review_text.span.decompose()
                    review_text = review_text.text.strip()
                    # 등록 날짜 찾기
                    date = row.select_one('td:nth-child(5)').get_text(strip=True)
                    # 한 행 출력
                    print(teacher_name, reg_num, review_text, date) 
                    # csv로 저장할 행 생성
                    review = [teacher_name, reg_num, review_text, date]
                    # 전체 리뷰에 추가
                    review_list.append(review)
        except Exception as e:
            

curr page: 1
정승제 선생님
정승제 선생님 4192 수능을 준비하며.. 2025.01.15
정승제 선생님 4191 생선님을 중3 때 알았더라면… 2025.01.15
정승제 선생님 4190 믿듣정 2025.01.14
정승제 선생님 4189 정승제가 갓승제인 이유 2025.01.14
정승제 선생님 4188 수강 후기 2025.01.14
정승제 선생님 4187 공통수학1 선생님 2025.01.13
정승제 선생님 4186 완벽한 강의 2025.01.13
정승제 선생님 4185 빈 개념 채우기 2025.01.13
정승제 선생님 4184 꽤 많은 도움이 됩니다 2025.01.11
정승제 선생님 4183 역시 갓승제 2025.01.11
정승제 선생님 4182 만족스러운 강의였습니다 2025.01.11
정승제 선생님 4181 언제 어디서든 집중해서 들을 수 있는 강의 2025.01.10
정승제 선생님 4180 1년동안 2025.01.10
정승제 선생님 4179 여러분이 도형에서 닮음을 못 찾는 이유? 2025.01.09
정승제 선생님 4178 chatgpt보다 빠른 질문 답변 2025.01.09
curr page: 2
정승제 선생님 4177 생선님 너무 완벽합니다 2025.01.09
정승제 선생님 4176 정말 좋은데요? 2025.01.08
정승제 선생님 4175 수강 후기 2025.01.08
정승제 선생님 4174 수학의 신 2025.01.08
정승제 선생님 4173 깔끔한 개념 설명 2025.01.07
정승제 선생님 4172 재밌다 2025.01.07
정승제 선생님 4171 기초도 모르면? 2025.01.07
정승제 선생님 4170 보는 재미가 열의와 열정을 만들어주는, 2025.01.04
정승제 선생님 4169 굿 2025.01.03
정승제 선생님 4168 감사합니다 2025.01.02
정승제 선생님 4167 강추해요 2025.01.02
정승제 선생님 4166 삼각함수 각변환 설명하시는 거 보고 충격받았어요 2025.01.01
정승제 선생님 4165 완강을 마

KeyboardInterrupt: 

In [41]:
# 파일로 저장
with open(csv_full_name, "w", encoding="utf-8", newline="") as csv_file:
    writer = csv.writer(csv_file)
    # 헤더 작성
    writer.writerow(["teacher_id", "no", "text", "date"])
    # 데이터 작성
    writer.writerows(review_list)